# Homework 03


## Part 1 - Questions

### Question 1: 

TODO (write the question)

* Who (population): TODO
* What (subject, discipline): TODO
* Where (location): TODO
* When (snapshot, longitudinal): TODO
* How much data do you need to do the analysis/work: TODO

### Question N: 

TODO (write the question)

* Who (population): TODO
* What (subject, discipline): TODO
* Where (location): TODO
* When (snapshot, longitudinal): TODO
* How much data do you need to do the analysis/work: TODO

### Who Might Collect Relevant Data / What Articles or Publications Cite a Relevant Data Set?

TODO (answer question here)

## Part 2 - Selecting a Data Set, Adding Documentation



1. Name / Title: (TODO name of data set)
2. Link to Data: (TODO link to any documentation about the data that you've found )
3. Source / Origin: 
	* Author or Creator: TODO
	* Publication Date: TODO
	* Publisher: TODO
	* Version or Data Accessed: TODO
4. License: (TODO name of license)
5. Can You Use this Data Set for Your Intended Use Case? (TODO answer this question)

## Format




### Overview

* Format: (TODO add what file format the data is in)
* Size: (TODO how large is the file in KB, MB, GB, etc. ... use finder, windows explorer for this)
* Number of Records: (TODO how many rows)



### Sample of Data




In [1]:
# TODO show a few lines of data from the actual file

# Use "regular" Python to do this in this code block
# Assume that jupyter-lab was started in your root directory
# with open('./data/raw/example-data.csv', 'r'): 


### Fields or Column Headers

* Field/Column 1: (TODO add field name and potential type using Python types)
* Field/Column 2: (TODO same as above)
* Field/Column N: (TODO same as above)

## Part 3 - Extract / Transform

TODO describe your process for extracting, transforming, cleaning your incoming data:

* what columns you'll be using (extracting) from your dataset
* how you're planning to convert the data into analogous python types / how you'll deal with type issues (for example, you may want a column that's numeric, but the data contains a label, such as 4 months) 
* any other kind of cleaning (_normalizing_ data, filling in missing values, etc.) that you'll have to perform


In [2]:
# TODO write code here!


## Part 4 - Descriptive Statistics

### Analysis on Numeric Data

#### Central Tendency



In [3]:
# TODO: copy and paste your results here

#### Dispersion

In [4]:
# TODO: copy and paste your results here

#### Outliers


In [5]:
# TODO: copy and paste your results here

#### Other


In [6]:
# TODO: copy and paste your results here

###  Analysis on Categorical Data

#### Frequency

In [7]:
# TODO: copy and paste your results here

#### Unique Values

In [8]:
# TODO: copy and paste your results here

## Part 5 - Visualizations



In [9]:
# TODO: add visualizations here

TODO Describe the visualizations in the previous cell


## Part 6 - Conclusion

TODO: write your conclusion here (interpret results of calculations; does it help answer your original questions?)